In [0]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
park_df = pd.read_csv('/content/drive/My Drive/AIML_NoteBooks/LabAndAssignments/parkinsons.data');
park_df.head()


,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,0.02182,0.03130,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,0.03134,0.04518,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,0.02757,0.03858,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,0.02924,0.04005,0.03772,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,0.03490,0.04825,0.04465,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [0]:
for column in park_df.columns:
    if park_df[column].dtype == type(object):
        le = preprocessing.LabelEncoder()
        park_df[column] = le.fit_transform(park_df[column])

In [5]:
park_df['status'].value_counts()

1    147
0     48
Name: status, dtype: int64

In [0]:
park_df['status'] = park_df['status'].replace({0: 'Healthy', 1: 'PD'})

In [0]:
X = park_df.drop("status", axis=1)
Y = park_df['status']

In [0]:
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3,random_state=0)

In [0]:
Xtrain=np.array(xtrain)
Ytrain=np.array(ytrain)

In [0]:
Xtest=np.array(xtest)
Ytest=np.array(ytest)

In [11]:
model=DecisionTreeClassifier(max_depth=4,criterion='entropy',random_state=0)
model.fit(Xtrain,Ytrain)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [0]:
ypred=model.predict(Xtest)

In [13]:
acc=metrics.accuracy_score(Ytest,ypred)
print(acc)

0.9830508474576272


In [14]:
cm=metrics.confusion_matrix(Ytest,ypred)
print(cm)

[[13  0]
 [ 1 45]]


In [15]:
cr=metrics.classification_report(Ytest,ypred)
print(cr)

              precision    recall  f1-score   support

     Healthy       0.93      1.00      0.96        13
          PD       1.00      0.98      0.99        46

    accuracy                           0.98        59
   macro avg       0.96      0.99      0.98        59
weighted avg       0.98      0.98      0.98        59



In [16]:
print (pd.DataFrame(model.feature_importances_, columns = ["Imp"], index = xtrain.columns))

                       Imp
name              0.272546
MDVP:Fo(Hz)       0.000000
MDVP:Fhi(Hz)      0.094626
MDVP:Flo(Hz)      0.090233
MDVP:Jitter(%)    0.000000
MDVP:Jitter(Abs)  0.000000
MDVP:RAP          0.000000
MDVP:PPQ          0.000000
Jitter:DDP        0.000000
MDVP:Shimmer      0.000000
MDVP:Shimmer(dB)  0.000000
Shimmer:APQ3      0.000000
Shimmer:APQ5      0.000000
MDVP:APQ          0.000000
Shimmer:DDA       0.000000
NHR               0.000000
HNR               0.000000
RPDE              0.032667
DFA               0.115632
spread1           0.073318
spread2           0.000000
D2                0.000000
PPE               0.320979


In [18]:
#Perform Grid Search Method to find the optimal max_depth size
from sklearn.model_selection import GridSearchCV
parameter={'max_depth':np.arange(1,10),'criterion' : ['entropy','gini']}
GS=GridSearchCV(model,parameter,cv=3)
GS.fit(X,Y)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='entropy', max_depth=4,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=0,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
     

In [19]:
GS.score(Xtest,Ytest)

0.8983050847457628

In [0]:
gs_pred=GS.predict(Xtest)

In [21]:
metrics.accuracy_score(Ytest,gs_pred)

0.8983050847457628

In [22]:
GS.best_params_

{'criterion': 'entropy', 'max_depth': 1}